In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import tensorflow as tf
from keras.utils import pad_sequences
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from transformers import TFAutoModel, TFXLMRobertaModel

class TargetedHSD:
    def __init__(self, model_path = None, tokenizer_path = None):
        if not model_path:
            self.__model_path = '../saved_model/bigrulstmcnn_xlmr2.h5'
        else:
            self.__model_path = model_path
        if not tokenizer_path:
            self.__tokenizer_path = 'xlm-roberta-base'
        else:
            self.__tokenizer_path = tokenizer_path
        
        self._tokenizer = AutoTokenizer.from_pretrained(self.__tokenizer_path)
        self._model = tf.keras.models.load_model(self.__model_path, custom_objects={'TFXLMRobertaModel': TFXLMRobertaModel})
        self.result = None
        self.orginal_label = None
    
    def predict(self, text):
        # encoded_text = self._tokenizer.texts_to_sequences([text])
        # encoded_text = pad_sequences(encoded_text, maxlen=100, padding='post')
        encoded_text = np.array(self._tokenizer([text], max_length=100, padding='max_length', truncation=True)['input_ids'])
        encoded_text = {
            "input_ids": np.asarray(self._tokenizer([text], max_length=50, padding='max_length', truncation=True)['input_ids']),
            "attention_mask": np.asarray(self._tokenizer([text], max_length=50, padding='max_length', truncation=True)['attention_mask'])
        }
        pred = self._model.predict(encoded_text)
        pred = np.argmax(pred.reshape(-1, 5, 4), axis=-1)

        # y_test_pred_new = []
        # for y in pred:
        #     lb = []
        #     for i in range(0, len(y)):
        #         if y[i] >= 0.5:
        #             lb.append(1)
        #         else:
        #             lb.append(0)
        #     y_test_pred_new.append(lb)
        self.orginal_label = pred[0]
    
    def return_label(self):
        true_labels = []
        TYPE = {
            1: "clean",
            2: "offensive",
            3: "hate"
        }
        LABEL = {
            0: "individual",
            1: "groups",
            2: "religion",
            3: "race",
            4: "politics"
        }
        # LABEL = [('individual', 1),
        #             ('individual', 2),
        #             ('individual', 3),
        #             ('groups', 1),
        #             ('groups', 2),
        #             ('groups', 3),
        #             ('religion/creed', 1),
        #             ('religion/creed', 2),
        #             ('religion/creed', 3),
        #             ('race/ethnicity', 1),
        #             ('race/ethnicity', 2),
        #             ('race/ethnicity', 3),
        #             ('politics', 1),
        #             ('politics', 2),
        #             ('politics', 3)
        #         ]
        print(self.orginal_label)
        for i in range(0, len(self.orginal_label)):
            if self.orginal_label[i] > 0:
                t = LABEL[i] + "#" + TYPE[int(self.orginal_label[i])]
                true_labels.append(t)

        self.result = true_labels
        return true_labels 

/home/bakansm/.conda/envs/tensorflow/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
cls = TargetedHSD()

2023-04-06 18:56:17.067633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-06 18:56:17.067662: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-06 18:56:17.067689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BakanPC): /proc/driver/nvidia/version does not exist
2023-04-06 18:56:17.067964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 18:56:19.010574: W tensorflow/tsl/framework/cpu_allocator_impl.cc

KeyboardInterrupt: 

In [ ]:

dev = 

In [ ]:
# Nhap vao day
# text = "Nếu mà ai thích tiền, thích hào quang thì cứ lên đây để nếm thử 4 chữ hào quang rực rỡ đi rồi biết nó là cái gì!"
text = "Bông lan trứng muối"

In [ ]:
cls.predict(text)

1/1 [==============================] - 0s 499ms/step


In [ ]:
cls.return_label()

[1 0 0 0 0]


['individual#clean']